In [352]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import style
import seaborn as sns
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import graphviz as gr
from linearmodels.iv import IV2SLS

%matplotlib inline

pd.set_option("display.max_columns", 5)
style.use("fivethirtyeight")

In [353]:
data = pd.read_csv("../Casual-Inference/data/income_data/train.csv")
data = data.dropna()
data.describe()
data

,age,workclass,...,native-country,income_>50K
0,67,Private,...,United-States,1
1,17,Private,...,United-States,0
2,31,Private,...,United-States,1
3,58,State-gov,...,United-States,0
4,25,State-gov,...,United-States,0
...,...,...,...,...,...
43952,52,Private,...,United-States,1
43953,19,Private,...,United-States,0
43954,30,Private,...,United-States,0
43955,46,Private,...,United-States,0


In [354]:
data = data.dropna(axis = 0)
data = data.loc[:, ["race", "educational-num", "age", "income_>50K", "occupation", "relationship", "marital-status", "native-country", "gender", "capital-gain", "capital-loss", "hours-per-week", "workclass"]]
data.rename(columns = {'educational-num':'educational_num', "income_>50K": "income_bigger_than_50K", 'marital-status':'marital_status'}, inplace = True)
data["race"] = data["race"].replace(to_replace ="Amer-Indian-Eskimo",
                 value ="Indian")
data["race"] = data["race"].replace(to_replace ="Asian-Pac-Islander",
                 value ="Asian")
occupationDict = {
    "Exec-managerial" : 0,
    "Other-service" : 1,
    "Transport-moving" : 2,
    "Adm-clerical" : 3,
    "Machine-op-inspct" : 4,
    "Sales" : 5,
    "Handlers-cleaners" : 6,
    "Farming-fishing" : 7,
    "Protective-serv" : 8,
    "Prof-specialty" : 9,
    "Craft-repair" : 10,
    "Tech-support" : 11,
    "Priv-house-serv" : 12,
    "Armed-Forces" : 13
}
raceDict = {
    "White" : 0,
    "Black" : 1,
    "Asian" : 2,
    "Indian" : 3,
    "Other" : 4
}
genderDict = {
    "Male" : 0,
    "Female" : 1
}
maritalDict = {
    "Divorced":0,
    "Never-married":1,
    "Married-civ-spouse":2,
    "Widowed":3,
    "Separated":4,
    "Married-spouse-absent":5,
    "Married-AF-spouse":6
}
countryDict = {
    'United-States': 0,
    'Japan': 1,
    'South': 2,
    'Portugal': 3,
    'Italy': 4,
    'Mexico': 5,
    'Ecuador': 6,
    'England':7 ,
    'Philippines': 8,
    'China': 9,
    'Germany': 10,
    'Dominican-Republic': 11,
    'Jamaica': 12,
    'Vietnam': 13,
    'Thailand': 14,
    'Puerto-Rico': 15,
    'Cuba': 16,
    'India': 17,
    'Cambodia': 18,
    'Yugoslavia': 19,
    'Iran': 20,
    'El-Salvador': 21,
    'Poland': 22,
    'Greece': 23,
    'Ireland': 24,
    'Canada': 25,
    'Guatemala': 26,
    'Scotland': 27,
    'Columbia': 28,
    'Outlying-US(Guam-USVI-etc)': 29,
    'Haiti': 30,
    'Peru': 31,
    'Nicaragua': 32,
    'Trinadad&Tobago': 33,
    'Laos': 34,
    'Taiwan': 35,
    'France': 36,
    'Hungary': 37,
    'Honduras': 38,
    'Hong': 39,
    'Holand-Netherlands': 40
}
workclassDict = {
    'Private': 0,
    'State-gov': 1,
    'Self-emp-not-inc': 2,
    'Federal-gov': 3,
    'Local-gov': 4,
    'Self-emp-inc': 5,
    'Without-pay': 6
}
def map_age(age):
    if age < 20 or age > 90: return -1
    if 20 <= age < 25: return 1
    if 25 <= age < 30: return 2
    if 30 <= age < 35: return 3
    if 35 <= age < 40: return 4
    if 40 <= age < 45: return 5
    if 45 <= age < 50: return 6
    if 50 <= age < 60: return 7
    if 60 <= age < 75: return 8
    if 75 <= age < 90: return 9

data["occupation"] = data["occupation"].map(occupationDict)
data["race"] = data["race"].map(raceDict)
data["gender"] = data["gender"].map(genderDict)
data["age_interval"] = data["age"].map(map_age)
data["marital_status"] = data["marital_status"].map(maritalDict)
data["native-country"] = data["native-country"].map(countryDict)
data["workclass"] = data["workclass"].map(workclassDict)
data["educational_num"].unique()

array([16,  8, 13,  4, 10,  9,  5,  6,  7, 14,  1,  3, 15, 11, 12,  2])

In [355]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from tensorflow import keras

In [356]:
X = data[['educational_num', 'age', 'race', 'gender', 'marital_status', 'native-country', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass']]
X

,educational_num,age,...,hours-per-week,workclass
0,16,67,...,60,0
1,8,17,...,15,0
2,13,31,...,40,0
3,4,58,...,40,1
4,10,25,...,40,1
...,...,...,...,...,...
43952,13,52,...,50,0
43953,9,19,...,40,0
43954,10,30,...,58,0
43955,13,46,...,35,0


In [357]:
y = data['occupation']
y

0        0
1        1
2        0
3        2
4        1
        ..
43952    0
43953    1
43954    5
43955    5
43956    6
Name: occupation, Length: 40727, dtype: int64

In [358]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.15)

In [359]:
encoder=LabelEncoder()
y_train_encoded=encoder.fit_transform(y_train)
y_val_encoded=encoder.fit_transform(y_val)
y_test_encoded=encoder.fit_transform(y_test)

y_train_cat=to_categorical(y_train_encoded)
y_val_cat=to_categorical(y_val_encoded)
y_test_cat=to_categorical(y_test_encoded)


In [360]:
network = Sequential()
network.add(Dense(units=128, activation='relu'))
network.add(Dense(units=128, activation='relu'))
network.add(Dense(units=64, activation='relu'))
network.add(Dense(units=14, activation='softmax'))
opt=keras.optimizers.Adam(learning_rate=0.01,decay=0.0005)
network.compile(
  optimizer = opt,
  loss = 'categorical_crossentropy', 
  metrics = ['accuracy']
)

network.fit(X_train,y_train_cat,validation_data=(X_val,y_val_cat),epochs=200)
test_loss, test_acc = network.evaluate(X_test,y_test_cat)

print('The loss function on our test set: ' + str(test_loss))
print('Accuracy on our test set: ' + str(test_acc))

Epoch 1/200
974/974 [==============================] - 2s 2ms/step - loss: 3.9293 - accuracy: 0.2076 - val_loss: 2.1706 - val_accuracy: 0.2490
Epoch 2/200
974/974 [==============================] - 1s 2ms/step - loss: 2.1584 - accuracy: 0.2550 - val_loss: 2.1825 - val_accuracy: 0.2397
Epoch 3/200
974/974 [==============================] - 1s 1ms/step - loss: 2.1006 - accuracy: 0.2672 - val_loss: 2.0619 - val_accuracy: 0.2704
Epoch 4/200
974/974 [==============================] - 1s 1ms/step - loss: 2.0593 - accuracy: 0.2847 - val_loss: 2.0378 - val_accuracy: 0.2884
Epoch 5/200
974/974 [==============================] - 1s 1ms/step - loss: 2.0310 - accuracy: 0.2945 - val_loss: 2.0261 - val_accuracy: 0.2851
Epoch 6/200
974/974 [==============================] - 1s 1ms/step - loss: 2.0137 - accuracy: 0.2989 - val_loss: 2.0127 - val_accuracy: 0.2975
Epoch 7/200
974/974 [==============================] - 1s 1ms/step - loss: 2.0042 - accuracy: 0.3038 - val_loss: 2.0286 - val_accuracy: 0.2944